In [4]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *

In [5]:
# Read station catalogue file
root_folder = r"Z:\PhD_Datasets&Analysis"
gdb = root_folder + "\\ArcGIS_Pro_Project\\Default.gdb"
fc = gdb + "\\Stations_Catalogue" 

In [ ]:
hydrosheds = root_folder + "\\Info_Inputs\HydroSHEDS"
flow_dir_folder = hydrosheds + "\\FlowDirection"
flow_acc_folder = hydrosheds + "\\FlowAccumulation"

# Set overwrite option
env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

In [ ]:
# Make a layer from the feature class
arcpy.management.MakeFeatureLayer(fc, "selected_station")

with arcpy.da.SearchCursor(fc, ["OBJECTID", "BASIN", "CONTINENT"]) as cursor:

    for row in cursor:

        print("Processing basin: " + row[1])

        # Select each station
        print("\tSelecting corresponding station")
        arcpy.SelectLayerByAttribute_management("selected_station", "NEW_SELECTION", "OBJECTID = {}".format(row[0]))
        
        snap_distance = 0.0046296296 #Change according to spatial resolution (approx. 500 m)     
        Extent = "MAXOF"
        fdir_name = row[2] + "_dir_3s"
        facc_name = row[2] + "_acc_3s"
        fdir_grid = flow_dir_folder + "\\" + fdir_name + "\\" + fdir_name +'.tif'
        facc_grid = flow_acc_folder + "\\" + facc_name + "\\" + facc_name +'.tif'
        snapped_pour_point = root_folder + "\\Info_Inputs\\Streamflow_Sts_Drainage_Areas\\snapped_pour_point_" + row[1] + ".shp"
        watershed = root_folder + "\\Info_Inputs\\Streamflow_Sts_Drainage_Areas\\Cathment_" + row[1] + ".shp"

        # Process: Snap Pour Point (Snap Pour Point) (sa)
        with arcpy.EnvManager(extent=Extent):
            print("\tSnapping pour point")
            pour_point_raster = arcpy.sa.SnapPourPoint("selected_station", facc_grid, snap_distance, "OBJECTID")

        print("\tConverting snapped pour raster to point")
        # Process: Raster to Point (Raster to Point) (conversion)
        arcpy.conversion.RasterToPoint(pour_point_raster, snapped_pour_point, "VALUE")

        print("\tDelineating watershed")
        # Process: Watershed (Watershed) (sa)
        watershed_raster = arcpy.sa.Watershed(fdir_grid, pour_point_raster, "VALUE")

        print("\tConverting watershed raster to polygon")
        # Process: Raster to Polygon (Raster to Polygon) (conversion)
        arcpy.conversion.RasterToPolygon(watershed_raster, watershed, "SIMPLIFY", "VALUE", "SINGLE_OUTER_PART", None)

print("DONE!!")